In [25]:
!pip install pandas

In [26]:
import numpy as np

In [27]:
import pandas as pd

In [30]:
from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import hi_stopwords,en_stopwords
from texthero.preprocessing import remove_digits
import pandas as pd

In [31]:
from sentence_transformers import SentenceTransformer

In [32]:
from torch.nn import CosineSimilarity
import torch

In [33]:
import pickle

In [34]:
cd E:\Dinesh

E:\Dinesh


In [35]:
pwd

'E:\\Dinesh'

In [9]:
df = pd.read_excel("din.xlsx")
df

,Q,A
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...
1,Have feedback or want to know more?,We are a passionate set of application focused...
2,What components does kandi cover?,kandi helps you select software components acr...
3,How do I use kandi?,kandi provides two simplified experiences to h...
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...
5,How do I implement the components that I have ...,The component listing and detailed insights pa...


In [37]:
df = pd.read_excel("din.xlsx")
df['clean_Q'] = df['Q'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 


In [38]:
df

,Q,A,clean_Q
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...,What is kandi
1,Have feedback or want to know more?,We are a passionate set of application focused...,Have feedback or want to know more
2,What components does kandi cover?,kandi helps you select software components acr...,What components does kandi cover
3,How do I use kandi?,kandi provides two simplified experiences to h...,How do I use kandi
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...,How do I shortlist components on kandi
5,How do I implement the components that I have ...,The component listing and detailed insights pa...,How do I implement the components that I have ...


In [39]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [40]:
q_embs = model.encode(df['clean_Q'], convert_to_tensor=True)

C:\Users\DINESH\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [41]:
q_embs.shape

torch.Size([6, 384])

In [42]:
usr_query = "Tell me anout kandi"

In [45]:
df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
df_usr_query

,usr_query
0,Tell me anout kandi


In [46]:
df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 
df_usr_query

,usr_query,clean_usr_Q
0,Tell me anout kandi,Tell me anout kandi


In [47]:
usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
usr_q_emb.shape

C:\Users\DINESH\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


torch.Size([1, 384])

In [48]:
import torch


In [49]:
cos_fn = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

In [50]:
cos_fn(usr_q_emb, q_embs)

tensor([0.6839, 0.1686, 0.5642, 0.6428, 0.4877, 0.4598])

In [51]:
idx = np.argmax(cos_fn(usr_q_emb, q_embs)).item()

In [52]:
df['A'][idx]

'kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.'

In [53]:
def pred_answer(user_query):
    df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
    df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 
    usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
    cos_fn = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    idx = np.argmax(cos_fn(usr_q_emb, q_embs)).item()
    return df['A'][idx]
    

In [ ]:
while True:
    usr_query = input("Ask a query: ")
    if usr_query == "exit":
        break
    else:
        print(pred_answer(usr_query))
        print("---------------")

Ask a query: Tell me about kandi
kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.
---------------
Ask a query: How can we use kandi
kandi provides two simplified experiences to help you choose the right software component to accelerate your application development:

1. Search
You can search for the component using natural language to describe your functional and technical requirements, and kandi gets to work by matching these over 430 million knowledge items to show you a shortlist.
You can further filter them or refine your query and pick your chosen ones based on scores available on the component listing page.
Click on the components from the list to review detailed insights such as support, quality, security, and a reference guide covering code snippets, community discussions from the provider, and popular channels.
The component listing and detailed